## Overview of the workflow for the EDA and Graph

### EDA of the meta-analytic analysis of the GNPC data, exploring the summary statistics and also down the line we are building the graph from the gathered PPI network information through STRING database 


In [41]:
import pandas as pd
import torch
import os
import numpy as np
from torch_geometric.data import Data


torch.manual_seed(42)

import pandas as pd
import numpy as np

In [42]:
#loading the GNPC supplementary table 5 for the summary statistics 
file_path = '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/01-raw/gnpc_supp.xlsx'
sheet_name = 'SuppTbl5'

In [43]:
# Read the Excel file, skipping the first row (header=1)

try:
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)
    print("DataFrame shape:", df.shape)
    print(df.head())
    print(df.columns)
except Exception as e:
    print("Error reading Excel file:", e)

DataFrame shape: (7289, 75)
      SeqId    SomaId                                     TargetFullName  \
0  10000-28  SL019233                                 Beta-crystallin B2   
1   10001-7  SL002564  RAF proto-oncogene serine/threonine-protein ki...   
2  10003-15  SL019245                             Zinc finger protein 41   
3  10006-25  SL019228                ETS domain-containing protein Elk-1   
4  10008-43  SL019234              Guanylyl cyclase-activating protein 1   

  Target UniProt EntrezGeneID EntrezGeneSymbol Organism  \
0  CRBB2  P43320         1415           CRYBB2    Human   
1  c-Raf  P04049         5894             RAF1    Human   
2  ZNF41  P51814         7592            ZNF41    Human   
3   ELK1  P19419         2002             ELK1    Human   
4  GUC1A  P43080         2978           GUCA1A    Human   

   Avg_StdBeta_weighted_AD  Meta_p_weighted_AD  ...  FTD_StdBeta_I   FTD_p_I  \
0                 0.020411            0.105892  ...       0.006329  0.928649   


In [44]:
# Define the required columns and their new names

required_columns = {
    'EntrezGeneSymbol': 'GeneSymbol',
    'Avg_StdBeta_weighted_AD': 'AD_beta',
    'Meta_p_weighted_AD': 'AD_p',
    'Avg_StdBeta_weighted_PD': 'PD_beta',
    'Meta_p_weighted_PD': 'PD_p',
    'Avg_StdBeta_weighted_FTD': 'FTD_beta',
    'Meta_p_weighted_FTD': 'FTD_p',
    'StdBeta_ALS': 'ALS_beta', # This is the non-weighted one for ALS
    'p_ALS': 'ALS_p'
}

missing_cols = [col for col in required_columns.keys() if col not in df.columns]
if missing_cols:
    print(f"Error: The following required columns are missing: {missing_cols}")
else:
    # Select and rename the columns
    features_df = df[list(required_columns.keys())].copy() 
    features_df.rename(columns=required_columns, inplace=True)
    
    print("New DataFrame shape:", features_df.shape)
    print(features_df.head())

New DataFrame shape: (7289, 9)
  GeneSymbol   AD_beta      AD_p   PD_beta      PD_p  FTD_beta     FTD_p  \
0     CRYBB2  0.020411  0.105892 -0.019541  0.002026  0.016847  0.126806   
1       RAF1 -0.018095  0.863149  0.004977  0.189185  0.007533  0.393355   
2      ZNF41  0.049706  0.000772  0.004067  0.466674  0.000385  0.385409   
3       ELK1  0.028990  0.002776  0.010033  0.580357  0.000471  0.457739   
4     GUCA1A -0.014837  0.151340 -0.006160  0.018256 -0.054077  0.095542   

   ALS_beta     ALS_p  
0 -0.075116  0.201204  
1 -0.052058  0.369734  
2  0.015578  0.790375  
3  0.088028  0.129374  
4  0.030428  0.603799  


In [45]:
# Pre-processing

features_df['GeneSymbol'].fillna('UNKNOWN', inplace=True)
features_df['GeneSymbol'] = features_df['GeneSymbol'].astype(str)

# Some gene symbols might be "GENE1;GENE2". We take the first one.
features_df['GeneSymbol'] = features_df['GeneSymbol'].apply(lambda x: x.split(';')[0])

# Check for duplicates 
print(f"\nNumber of unique gene symbols: {features_df['GeneSymbol'].nunique()}")
print(f"Total number of rows: {len(features_df)}")
# If there are duplicates, we will keep the first occurrence
features_df.drop_duplicates(subset='GeneSymbol', keep='first', inplace=True)
print(f"Number of rows after dropping duplicates: {len(features_df)}")

# Set GeneSymbol as the index
features_df.set_index('GeneSymbol', inplace=True)

beta_cols    = ['AD_beta', 'PD_beta', 'FTD_beta', 'ALS_beta']
p_value_cols = ['AD_p', 'PD_p', 'FTD_p', 'ALS_p']

print("\nMissing values per column before cleaning:")
print(features_df[beta_cols + p_value_cols].isnull().sum())

# Betas: missing → 0 (no effect)
features_df[beta_cols] = features_df[beta_cols].fillna(0.0)

# p-values: missing → 1 (non-significant)
features_df[p_value_cols] = features_df[p_value_cols].fillna(1.0)

print("\nMissing values after cleaning:")
print(features_df[beta_cols + p_value_cols].isnull().sum())



Number of unique gene symbols: 6386
Total number of rows: 7289
Number of rows after dropping duplicates: 6386

Missing values per column before cleaning:
AD_beta     0
PD_beta     0
FTD_beta    0
ALS_beta    0
AD_p        0
PD_p        0
FTD_p       0
ALS_p       0
dtype: int64

Missing values after cleaning:
AD_beta     0
PD_beta     0
FTD_beta    0
ALS_beta    0
AD_p        0
PD_p        0
FTD_p       0
ALS_p       0
dtype: int64


## Feature Engineering 

We are building a feature matrix from the GNPC's summarys statistics

- `*_beta` columns has the standardized effect size for each disease 
  (case vs control: AD, PD, FTD, ALS)
  
- `*_logp` columns has the transformed p-values, computed as `-log10(p + ε)` to have usable numbers 


We keep only these 8 summary-statistic features:

- AD_beta, AD_logp  
- PD_beta, PD_logp  
- FTD_beta, FTD_logp  
- ALS_beta, ALS_logp  

These will be the basis of our graph model:

- The **betas** serve as the supervised targets (what we try to predict later on)
- The **–log10(p)** values are part of the input feature vector, encoding the strength  
  of evidence that each protein is dysregulated in each disease

In [46]:
# Define a small constant to add to p-values to avoid log(0)
epsilon = 1e-300

# Creating the -log10(p-value) features from the raw p-value columns
p_value_cols = ['AD_p', 'PD_p', 'FTD_p', 'ALS_p']
for col in p_value_cols:
    new_col_name = col.replace('_p', '_logp')  # example: AD_p -> AD_logp
    features_df[new_col_name] = -np.log10(features_df[col] + epsilon)

print("\nDataFrame after adding -log10(p) features:")
print(features_df.head())

# Defining now which columns should go into protein_features.csv
# We keep only betas + logp, no raw p-values from the summarys statistics
final_feature_order = [
    'AD_beta',  'AD_logp',
    'PD_beta',  'PD_logp',
    'FTD_beta', 'FTD_logp',
    'ALS_beta', 'ALS_logp',
]

# checking if all the columns exist 
missing = [c for c in final_feature_order if c not in features_df.columns]
if missing:
    raise ValueError(f"Missing expected columns for protein_features.csv: {missing}")

final_features_df = features_df[final_feature_order]

print("\nFinal, ordered feature DataFrame:")
print(final_features_df.head())

# Save

output_dir = '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/02-preprocessed'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'protein_features.csv')
final_features_df.to_csv(output_path)

print(f"\nCreated and saved the feature matrix to '{output_path}'")
print(f"Final shape of the feature matrix: {final_features_df.shape}")



DataFrame after adding -log10(p) features:
             AD_beta      AD_p   PD_beta      PD_p  FTD_beta     FTD_p  \
GeneSymbol                                                               
CRYBB2      0.020411  0.105892 -0.019541  0.002026  0.016847  0.126806   
RAF1       -0.018095  0.863149  0.004977  0.189185  0.007533  0.393355   
ZNF41       0.049706  0.000772  0.004067  0.466674  0.000385  0.385409   
ELK1        0.028990  0.002776  0.010033  0.580357  0.000471  0.457739   
GUCA1A     -0.014837  0.151340 -0.006160  0.018256 -0.054077  0.095542   

            ALS_beta     ALS_p   AD_logp   PD_logp  FTD_logp  ALS_logp  
GeneSymbol                                                              
CRYBB2     -0.075116  0.201204  0.975136  2.693367  0.896860  0.696364  
RAF1       -0.052058  0.369734  0.063914  0.723114  0.405216  0.432111  
ZNF41       0.015578  0.790375  3.112235  0.330987  0.414078  0.102167  
ELK1        0.088028  0.129374  2.556576  0.236305  0.339382  0.888151  

In [47]:
# Create UniProt to Gene Symbol Mapping 

file_path = '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/01-raw/gnpc_supp.xlsx'
sheet_name = 'SuppTbl5'

# Load the Original GNPC Data to get the mapping 
original_df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)
print("Original GNPC table shape:", original_df.shape)

# Select only the columns we need for mapping and drop missing values
mapping_df = original_df[['UniProt', 'EntrezGeneSymbol']].dropna().copy()

# If a gene has multiple symbols separated by ';', keep the first one
mapping_df['EntrezGeneSymbol'] = (
    mapping_df['EntrezGeneSymbol']
    .astype(str)
    .apply(lambda x: x.split(';')[0])
)

# Ensure each UniProt appears only once
mapping_df = mapping_df.drop_duplicates(subset='UniProt')

# Create the dictionary: {UniProt_ID: GeneSymbol}
uniprot_to_gene_map = dict(zip(mapping_df['UniProt'], mapping_df['EntrezGeneSymbol']))

print(f"Created a mapping dictionary with {len(uniprot_to_gene_map)} entries.")
print("Example mappings:", list(uniprot_to_gene_map.items())[:5])


Original GNPC table shape: (7289, 75)
Created a mapping dictionary with 6387 entries.
Example mappings: [('P43320', 'CRYBB2'), ('P04049', 'RAF1'), ('P51814', 'ZNF41'), ('P19419', 'ELK1'), ('P43080', 'GUCA1A')]


### GNPC Summary Statistics → Feature Matrix

- We start from the GNPC meta-analysis table which contains **7,289 rows** and **75 columns** of summary statistics across four neurodegenerative diseases (AD, PD, FTD, ALS)

- After:
  - cleaning `EntrezGeneSymbol` (splitting on `;`, keeping the first symbol),
  - dropping ambiguous/missing symbols, and using unique gene symbols per row

  we end up with **6,386 unique genes** 

- For each gene, we keep:
  - Effect sizes (`*_beta`): AD_beta, PD_beta, FTD_beta, ALS_beta  
    → these are the **targets** our model tries to predict
  - Significance features (`*_logp`): –log10(p) for AD, PD, FTD, ALS  
    → these are the **input node features**

Conceptually, each node now represents **“how strongly and significantly this protein is dysregulated in each disease.”**


### Incorprated confidence scores from STRING

Adding the confidence score from the STRING for the protein-protein interactions 

In [48]:
base_dir = '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/01-raw'

nodes_file_path = os.path.join(base_dir, 'ppi_nodes_raw.csv')   # node_id,index,UniProt
edges_file_path = os.path.join(base_dir, 'ppi_edges_raw.csv')   # src,dst,weight,combined_score

print("Nodes path:", nodes_file_path)
print("Edges path:", edges_file_path)
print("Nodes exists?", os.path.exists(nodes_file_path))
print("Edges exists?", os.path.exists(edges_file_path))

nodes_raw = pd.read_csv(nodes_file_path)
edges_raw = pd.read_csv(edges_file_path)

print("Loaded nodes file:", nodes_raw.shape)
print("Loaded edges file:", edges_raw.shape)


Nodes path: /Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/01-raw/ppi_nodes_raw.csv
Edges path: /Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/01-raw/ppi_edges_raw.csv
Nodes exists? True
Edges exists? True
Loaded nodes file: (5505, 3)
Loaded edges file: (80102, 4)


### Mapping STRING from UniProt 


In [49]:
# Map STRING indices -> UniProt
idx_to_uniprot = nodes_raw.set_index('index')['UniProt'].to_dict()

edges_raw['protein1_uniprot'] = edges_raw['src'].map(idx_to_uniprot)
edges_raw['protein2_uniprot'] = edges_raw['dst'].map(idx_to_uniprot)

print("\nAfter mapping indices to UniProt:")
print(edges_raw[['src', 'dst', 'protein1_uniprot', 'protein2_uniprot']].head())

# Map UniProt -> GeneSymbol 

edges_raw['protein1'] = edges_raw['protein1_uniprot'].map(uniprot_to_gene_map)
edges_raw['protein2'] = edges_raw['protein2_uniprot'].map(uniprot_to_gene_map)

# Drop rows where translation failed
translated_edges_df = edges_raw.dropna(subset=['protein1', 'protein2']).copy()
print(f"\nAfter UniProt→GeneSymbol mapping: {len(translated_edges_df)} edges remain")


# Set of proteins that actually have GNPC features
valid_gene_symbols = set(features_df.index)

print(f"{len(valid_gene_symbols)} unique gene symbols to use as our node list")

# Now filter edges
filtered_edges_df = translated_edges_df[
    translated_edges_df['protein1'].isin(valid_gene_symbols) &
    translated_edges_df['protein2'].isin(valid_gene_symbols)
].copy()

print(f"Kept {len(filtered_edges_df)} out of {len(translated_edges_df)} edges.")

# Filtering by min confidence score:

min_weight = 0.7
pre_conf_count = len(filtered_edges_df)
filtered_edges_df = filtered_edges_df[filtered_edges_df['weight'] >= min_weight].copy()
print(f"Applied weight >= {min_weight}: kept {len(filtered_edges_df)} / {pre_conf_count} edges.")

# Remove self-loops
pre_self_loop_count = len(filtered_edges_df)
filtered_edges_df = filtered_edges_df[filtered_edges_df['protein1'] != filtered_edges_df['protein2']]
print(f"Removed {pre_self_loop_count - len(filtered_edges_df)} self-loops.")

# Deduplicate undirected edges, keeping the highest-weight interaction per pair
filtered_edges_df['pair'] = filtered_edges_df.apply(
    lambda r: tuple(sorted((r['protein1'], r['protein2']))),
    axis=1
)

filtered_edges_df = filtered_edges_df.sort_values('weight', ascending=False)
dedup_edges_df = filtered_edges_df.drop_duplicates(subset='pair', keep='first')

dedup_edges_df['protein1_clean'] = dedup_edges_df['pair'].apply(lambda p: p[0])
dedup_edges_df['protein2_clean'] = dedup_edges_df['pair'].apply(lambda p: p[1])

edges_final = dedup_edges_df[['protein1_clean', 'protein2_clean', 'weight']].rename(
    columns={'protein1_clean': 'protein1', 'protein2_clean': 'protein2'}
)

print(f"Removed duplicates; final edge count: {len(edges_final)}")

#save
edges_output_dir = '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/02-preprocessed'
os.makedirs(edges_output_dir, exist_ok=True)

clean_edges_output_path = os.path.join(edges_output_dir, 'protein_edges_clean_weighted.csv')
edges_final.to_csv(clean_edges_output_path, index=False)

print(f"\nSaved the cleaned, weighted edge list to '{clean_edges_output_path}'")



After mapping indices to UniProt:
   src   dst protein1_uniprot protein2_uniprot
0    0   904           P84085           O75154
1    0  1878           P84085           P18085
2    0  4898           P84085           P84077
3    0  1629           P84085           P53367
4    0  5307           P84085           P53365

After UniProt→GeneSymbol mapping: 79834 edges remain
6386 unique gene symbols to use as our node list
Kept 79834 out of 79834 edges.
Applied weight >= 0.7: kept 79834 / 79834 edges.
Removed 130 self-loops.
Removed duplicates; final edge count: 39381

Saved the cleaned, weighted edge list to '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/02-preprocessed/protein_edges_clean_weighted.csv'


/var/folders/_w/09t1lq652qd_803897fd_b7h0000gn/T/ipykernel_77452/3431976242.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup_edges_df['protein1_clean'] = dedup_edges_df['pair'].apply(lambda p: p[0])
/var/folders/_w/09t1lq652qd_803897fd_b7h0000gn/T/ipykernel_77452/3431976242.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup_edges_df['protein2_clean'] = dedup_edges_df['pair'].apply(lambda p: p[1])


### Building the protein–protein interaction graph for PyTorch Geometric

As next we are combining the GNPC summary statistics with the STRING PPI network into
a single `torch_geometric.data.Data` object:

- **Nodes:** proteins present in `protein_features.csv` (indexed by GeneSymbol)
- **Node features (`x`):** 4-dimensional vector of –log10(p) values  
  (`AD_logp, PD_logp, FTD_logp, ALS_logp`)
- **Targets (`y`):** 4-dimensional vector of disease-specific effect sizes  
  (`AD_beta, PD_beta, FTD_beta, ALS_beta`), used for multi-task regression
- **Edges (`edge_index`):** undirected protein–protein interactions derived from STRING,  
  filtered to proteins in our feature matrix, with self-loops and duplicates removed
- **Masks:** random train/validation/test splits over nodes (70% / 15% / 15%)

This `processed_graph.pt` file is the main input to our GAT/GCN models: it encodes  
both per-protein GNPC summary statistics and the known interaction structure from STRING.


In [50]:
print("\nBuilding PyG graph object from features + weighted edges")

# 1. Features: split into X (inputs) and Y (labels)
label_cols   = ['AD_beta', 'PD_beta', 'FTD_beta', 'ALS_beta']
feature_cols = ['AD_logp', 'PD_logp', 'FTD_logp', 'ALS_logp']

missing_label_cols = [c for c in label_cols if c not in features_df.columns]
missing_feat_cols  = [c for c in feature_cols if c not in features_df.columns]
if missing_label_cols or missing_feat_cols:
    raise ValueError(
        f"Missing columns.\n"
        f"  label_cols missing: {missing_label_cols}\n"
        f"  feature_cols missing: {missing_feat_cols}"
    )

X = torch.tensor(features_df[feature_cols].values, dtype=torch.float32)
Y = torch.tensor(features_df[label_cols].values,   dtype=torch.float32)

print("X shape (features):", X.shape)  # [num_nodes, 4]
print("Y shape (labels):  ", Y.shape)  # [num_nodes, 4]

# 2. Load weighted edge list
edges_df = pd.read_csv('/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/02-preprocessed/protein_edges_clean_weighted.csv')  # protein1, protein2, weight
print(f"Loaded {len(edges_df)} weighted edges.")

# 3. Create mapping from gene symbol to node index
gene_symbols = features_df.index.tolist()
gene_to_idx = {gene: i for i, gene in enumerate(gene_symbols)}
print(f"Created mapping for {len(gene_to_idx)} genes to integer indices.")

# Ensure all edges refer to known genes 
edges_df = edges_df[
    edges_df['protein1'].isin(gene_to_idx) &
    edges_df['protein2'].isin(gene_to_idx)
].copy()
print(f"Edges after checking the presence in gene_to_idx: {len(edges_df)}")

# 4. Build edge_index and edge_attr (confidence weights)
edge_index = torch.tensor([
    [gene_to_idx[p] for p in edges_df['protein1']],
    [gene_to_idx[p] for p in edges_df['protein2']],
], dtype=torch.long)

edge_weight = torch.tensor(edges_df['weight'].values, dtype=torch.float32)
edge_attr = edge_weight.view(-1, 1)  # shape [num_edges, 1]

print("edge_index shape:", edge_index.shape)
print("edge_attr shape: ", edge_attr.shape)


Building PyG graph object from features + weighted edges
X shape (features): torch.Size([6386, 4])
Y shape (labels):   torch.Size([6386, 4])
Loaded 39381 weighted edges.
Created mapping for 6386 genes to integer indices.
Edges after checking the presence in gene_to_idx: 39381
edge_index shape: torch.Size([2, 39381])
edge_attr shape:  torch.Size([39381, 1])


### STRING PPI Network → Cleaned and added the Weighted Edges

- We first loaded the raw STRING data:
  - Nodes: **5,505** entries (`node_id`, `index`, `UniProt`)
  - Edges: **80,102** raw interactions with columns (`src`, `dst`, `weight`, `combined_score`).
- Using the STRING node table, we map each edge’s integer indices (`src`, `dst`) to UniProt IDs, then use our GNPC-derived mapping to translate UniProt → GeneSymbol

After this:

- **79,834** edges can be mapped to *pairs* of valid gene symbols
- All of these edges connect proteins that also appear in the GNPC feature matrix (good intersection between GNPC and STRING)

We then:

1. Filter to high-confidence interactions using the STRING `weight` (already in [0, 1]):
   - Here, `weight ≥ 0.7` keeps **all** of the mapped edges, indicating that the supplied network is already high-confidence
2. Remove self-loops (A–A interactions), removing **130** such edges
3. Treat the graph as *undirected*:
   - Canonicalize each edge to `(min(protein1, protein2), max(...))`
   - Sort by weight and keep the highest-confidence edge per undirected pair

This yields **39,381** unique undirected, high-confidence edges, which we save as:

- `data/02-preprocessed/protein_edges_clean_weighted.csv`  
  (`protein1`, `protein2`, `weight`)

## Spliting nodes and assemble the PyTorch Geometric `Data` object

We randomly permute all proteins (nodes) and define:

- **70%** of nodes as the training set  
- **15%** as the validation set  
- **15%** as the test set  

For each split we create a boolean mask (`train_mask`, `val_mask`, `test_mask`) with:

- `True` for nodes belonging to that split  
- `False` otherwise  

These masks are used during training and evaluation so that:

- The model learns only from training nodes  
- Hyperparameters are tuned on validation nodes  
- Final performance is reported on held-out test nodes.


In [51]:
# 5. Create train/val/test masks
num_nodes = X.shape[0]
perm = torch.randperm(num_nodes)

train_end = int(0.7 * num_nodes)
val_end   = int(0.85 * num_nodes)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask   = torch.zeros(num_nodes, dtype=torch.bool)
test_mask  = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_end]]      = True
val_mask[perm[train_end:val_end]] = True
test_mask[perm[val_end:]]         = True

print(f"Training nodes:   {train_mask.sum().item()}")
print(f"Validation nodes: {val_mask.sum().item()}")
print(f"Test nodes:       {test_mask.sum().item()}")

Training nodes:   4470
Validation nodes: 958
Test nodes:       958


### We combine everything into a single PyTorch Geometric `Data` object:

- `x` → node feature matrix `X` (log p-values for AD/PD/FTD/ALS)  
- `y` → target matrix `Y` (effect sizes for AD/PD/FTD/ALS)  
- `edge_index` → graph connectivity (who interacts with whom)  
- `edge_attr` → edge confidence weights from STRING (1D edge features)  
- `train_mask`, `val_mask`, `test_mask` → node splits for training and evaluation  
- `gene_symbols` → list of gene names for later interpretation of embeddings and clusters  

Finally, we save this graph as `data/02-preprocessed/processed_graph.pt`, which is the
single input object used by all downstream GNN training and clustering scripts

In [52]:
# 6. Assemble final PyG Data object
graph_data = Data(
    x=X,
    edge_index=edge_index,
    edge_attr=edge_attr,      
    y=Y,
    train_mask=train_mask,
    val_mask=val_mask,
    test_mask=test_mask,
    gene_symbols=gene_symbols,
)

print("\nAssembled final PyG Data object:")
print(graph_data)

# 7. Save to the path used by your training script
output_file = '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/02-preprocessed/processed_graph.pt'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
torch.save(graph_data, output_file)

print(f"\nSaved graph data to '{output_file}'")


Assembled final PyG Data object:
Data(x=[6386, 4], edge_index=[2, 39381], edge_attr=[39381, 1], y=[6386, 4], train_mask=[6386], val_mask=[6386], test_mask=[6386], gene_symbols=[6386])

Saved graph data to '/Users/yasemindilarasucu/Desktop/Chem277-Team4-Project/data/02-preprocessed/processed_graph.pt'


### Final PyTorch Geometric Graph Object

Using the cleaned features and edges, we construct a PyG `Data` object:

- **Nodes:** 6,386 proteins (one per gene symbol)
- **Node features (`x`):** shape `[6386, 4]`
  - Columns: `[AD_logp, PD_logp, FTD_logp, ALS_logp]`
- **Targets (`y`):** shape `[6386, 4]`
  - Columns: `[AD_beta, PD_beta, FTD_beta, ALS_beta]`
- **Edges (`edge_index`):** shape `[2, 39,381]`
  - Each column is a pair of node indices (protein–protein interaction).
- **Edge attributes (`edge_attr`):** shape `[39,381, 1]`
  - Single scalar per edge: STRING confidence `weight` in [0, 1].
- **Masks:**
  - Training nodes: 4,470  
  - Validation nodes: 958  
  - Test nodes: 958  

We saved this graph as:

- `data/02-preprocessed/processed_graph.pt`

This graph is exactly the object used by the downstream Multi-Task GNN (GATv2) model

## What this preprocessing step prepares us to do

At the end of this analysis in the notebook we have:

- A **feature matrix** for 6,386 proteins:
  - Inputs (node features): `AD_logp`, `PD_logp`, `FTD_logp`, `ALS_logp`
    - how statistically strong the dysregulation is in each disease
  - Targets (labels): `AD_beta`, `PD_beta`, `FTD_beta`, `ALS_beta`
    - the direction and magnitude of the effect size per disease

- A **STRING-based protein–protein interaction graph**:
  - Nodes: the same 6,386 proteins that have GNPC summary statistics
  - Edges: 39,381 high-confidence STRING interactions
  - Edge attributes: STRING confidence scores in \[0, 1]

- A **PyTorch Geometric graph object**:
  - `x`: node features (4D logp vector per protein)
  - `y`: multi-task regression targets (4D beta vector per protein)
  - `edge_index` + `edge_attr`: PPI structure and confidence
  - train/val/test masks over nodes

--> How this connects to the main project question ?

Our main interest is **not** to “predict” effect sizes that we already know, but to use a graph neural network to learn network-aware embeddings of proteins that reflect:

1. How each protein is dysregulated across the four neurodegenerative diseases, and  
2. How it is positioned in the protein–protein interaction network

We use a GAT-based multi-task regression model as a **representation learner**:

- The loss (predicting `y` from `x`) forces the model to encode disease-specific dysregulation signals
- The graph structure (STRING PPIs) lets each protein update its representation based on its neighbors, using attention to prioritize the most informative interactions

After training, we extract the **learned protein embeddings** from the last GAT layer and:

- Use UMAP + HDBSCAN to cluster proteins into **modules**
- Characterize each module in terms of:
  - average effect sizes across diseases (AD/PD/FTD/ALS),
  - dominant disease patterns (AD-biased, PD-biased, mixed, pan-NDD),
  - and we can do pathway enrichment analysis for the interesting clusters we identified, this can be a future direction since the timing limit, but a high-level overview can be still great

--> this preprocessing pipeline sets up a graph where we can ask:

--> “Given known PPI relationships and GNPC multi-disease summary stats, what protein modules emerge as co-dysregulated across neurodegenerative diseases?”*


--> What we are adding with our model on top of the GNPC's already listed dysregulated proteins is: 

With the identified clusters in the embedding space, which gives us:

Not “a list of 6,386 dysregulated proteins” but **a map of the protein space where nearby proteins share both**:
similar multi-disease patterns and interaction context.
We’re basically organizing the GNPC signals into a network of modules that might be useful and can give insights to the diagnostics or therapuetics 

**Our Goal:** Identify network-defined plasma protein modules that are co-dysregulated across AD, PD, FTD, and ALS, using GNPC summary statistics + STRING PPIs

**What those modules represent:**
Patterns of co-dysregulation in the blood, not pure biological pathways in brain

**Likely mix of:**
systemic inflammation, vascular changes, metabolic changes, maybe some brain-derived proteins leaking out.. Eventually we are looking at the plasma which a blood sample, and proteins aren't expressed in the blood, so these protein's are coming from somewhere else, from nearby organs tissues..

With our project we are discovering plasma protein modules that show shared vs distinct patterns across NDDs, organized by protein–protein interactions

## At the end of this processing step, we have these files as an output:

**protein_features.csv** = clean per-protein GNPC summary stats (features + labels)

**protein_edges_clean_weighted.csv** = clean STRING PPI edges between those proteins

**processed_graph.pt** = the PyG graph built from those two, plus splits, ready for training/clustering